In [1]:
# ----------------------------------------------------
# --- Load packages
# ---------------------------------------------------- 
using UHDBindings 
import UHDBindings.LibUHD as LibUHD
import UHDBindings.initRxUHD
import UHDBindings.@assert_uhd

In [2]:
# ----------------------------------------------------
# --- Parameters 
# ---------------------------------------------------- 
carrierFreq = 868e6
samplingRate= 2e6
gain        = 25
uhdArgs     = ""
nbAntennaRx = 2

2

In [3]:
# ---------------------------------------------------- 
# --- Create radio 
# ---------------------------------------------------- 
# --- Open radio, in MIMO mode 
# Note the warning about the streamer. It can not be set in MIMO mode 
# The configuration will be updated later, and most of the parameters overwritten
# To ensure we have what we want, we need to bypass the streamer, and deactivate all antennas
radio = openUHD(carrierFreq,samplingRate,gain;nbAntennaRx=0,nbAntennaTx=0,bypassStreamer = true)


[INFO] [UHD] Mac OS; Clang version 12.0.0 (/home/mose/.julia/dev/BinaryBuilderBase/deps/downloads/llvm-project.git d28af7c654d8db0b68c175db5ce212d74fb5e9bc); Boost_107600; UHD_4.1.0.HEAD-0-g6bd0be9c
[INFO] [B200] Loading firmware image: /opt/local/share/uhd/images/usrp_b200_fw.hex...
[INFO] [B200] Detected Device: B200mini
[INFO] [B200] Loading FPGA image: /opt/local/share/uhd/images/usrp_b200mini_fpga.bin...
[INFO] [B200] Operating over USB 3.
[INFO] [B200] Initialize CODEC control...
[INFO] [B200] Initialize Radio control...
[INFO] [B200] Performing register loopback test... 
[INFO] [B200] Register loopback test passed
[INFO] [B200] Setting master clock rate selection to 'automatic'.
[INFO] [B200] Asking for clock rate 16.000000 MHz... 
[INFO] [B200] Actually got clock rate 16.000000 MHz.


UHDBinding(Base.RefValue{Ptr{UHDBindings.LibUHD.uhd_usrp}}(Ptr{UHDBindings.LibUHD.uhd_usrp} @0x000060000284fd80), UHDRx(UHDBindings.UHDRxWrapper(true, Ptr{UHDBindings.LibUHD.uhd_usrp} @0x000060000284fd80, Ptr{UHDBindings.LibUHD.uhd_rx_streamer} @0x0000600002764480, Ptr{UHDBindings.LibUHD.uhd_rx_metadata_t} @0x00006000003c7250, Base.RefValue{Ptr{UHDBindings.LibUHD.uhd_rx_streamer}}(Ptr{UHDBindings.LibUHD.uhd_rx_streamer} @0x0000600002764480), Base.RefValue{Ptr{UHDBindings.LibUHD.uhd_rx_metadata_t}}(Ptr{UHDBindings.LibUHD.uhd_rx_metadata_t} @0x00006000003c7250), Base.RefValue{UInt64}(0x0000000000000000)), 8.68e8, 2.0e6, 25, ["RX2"], [0], 0x0000000000000000, 0, 0, false), UHDTx(UHDBindings.UHDTxWrapper(true, Ptr{UHDBindings.LibUHD.uhd_usrp} @0x000060000284fd80, Ptr{UHDBindings.LibUHD.uhd_tx_streamer} @0x0000600002764990, Ptr{UHDBindings.LibUHD.uhd_tx_metadata_t} @0x0000600000b24280, Base.RefValue{Ptr{UHDBindings.LibUHD.uhd_tx_streamer}}(Ptr{UHDBindings.LibUHD.uhd_tx_streamer} @0x000060000

In [4]:
# ----------------------------------------------------
# --- Set up Radio 
# ---------------------------------------------------- 
# --- Channels  & Antenna 
channelIndexes = [0,1]
antennas       = ["TX/RX","RX2"]
# --- Streamer arguments
a1			   =  Base.unsafe_convert(Cstring,"fc32");
a2			   =  Base.unsafe_convert(Cstring,"sc16");
a3			   =  Base.unsafe_convert(Cstring,uhdArgs);
channel        = pointer(channelIndexes)
uhdArgs_0	   = LibUHD.uhd_stream_args_t(a1,a2,a3,channel,nbAntennaRx);

In [5]:
# --- RF Configuration 
for (c,currChan) in enumerate(channelIndexes[1:nbAntennaRx])
    # Set the carrier frequencies 
    updateCarrierFreq!(radio.rx,carrierFreq,currChan)
    # Set the sampling rate 
    updateSamplingRate!(radio.rx,samplingRate,currChan)
    # Set the gains 
    updateGain!(radio.rx,gain,currChan)
    # Set the antennas
    LibUHD.uhd_usrp_set_rx_antenna(radio.rx.uhd.pointerUSRP,antennas[c],currChan)
end

┌Rx Warning: Effective carrier frequency is 867.9999999999992 MHz and not 868.0 MHz
┌Rx Warning: Effective carrier frequency is 0.0 MHz and not 868.0 MHz
┌Rx Warning: Effective Rate is 0.0 MHz and not 2.0 MHz
┌Rx Warning: Effective gain is 0.0 dB and not 25 dB


[INFO] [B200] Asking for clock rate 32.000000 MHz... 
[INFO] [B200] Actually got clock rate 32.000000 MHz.


# References

- [ ] [Custom Use of LibUHD](https://docs.juliahub.com/UHDBindings/BKGJc/0.3.3/use_libuhd/)